In [35]:
import math

import numpy as np

## Data generation with showerpipe

In [1]:
import showerpipe as shp

Showerpipe wraps pythia8's generator in a standard Python iterator. To initialise, we need to pass it a settings file, see below:

In [2]:
%%file pythia-settings.cmnd
Beams:idB = -2212
Beams:eCM = 1960.
WeakSingleBoson:ffbar2gmZ = on
PhaseSpace:mHatMin = 80.
PhaseSpace:mHatMax = 120

Overwriting pythia-settings.cmnd


In [3]:
gen = shp.generator.PythiaGenerator("pythia-settings.cmnd")
gen

PythiaGenerator(xml_dir='/scratch/jlc1n20/mambaforge/envs/dgl/share/Pythia8/xmldoc')
├── Print
│   └── quiet = on
├── Random
│   ├── setSeed = on
│   └── seed = -1
├── Beams
│   ├── idB = -2212
│   └── eCM = 1960.
├── WeakSingleBoson
│   └── ffbar2gmZ = on
└── PhaseSpace
    ├── mHatMin = 80.
    └── mHatMax = 120

The string and developer representations of the object shows the settings used to instantiate it, as well as the location of the XML directory where Pythia8's data is stored. Events can be generated simply by looping over `gen`, or calling `next()`:

The string representation of `PythiaEvent` objects shows the length, whereas the developer representation exposes the thinly wrapped pythia8.Event object and address.

In [4]:
event = next(gen)
event

PythiaEvent(_event=<pythia8.Event object at 0x2b87f77917f0>)

In [5]:
print(event)

PythiaEvent(len=300)


`PythiaEvent` objects expose the generated data via numpy arrays, accessed with object attributes. Comprehensive documentation can be seen by either calling `help(event)`, or checking the [API reference online](https://showerpipe.readthedocs.io/en/latest/api/showerpipe.generator.PythiaEvent.html).

Some arrays are structured, such as `.pmu` (momentum), `.color`, and `.edges`. These are just views over the data, the underlying array is contiguous and unstructured.

In [6]:
event.pmu[:8]  # momenta of the first 8 particles

array([( 0.00000000e+00,  0.00000000e+00,  979.99955084, 980.        ),
       ( 0.00000000e+00,  0.00000000e+00, -979.99955084, 980.        ),
       ( 0.00000000e+00,  0.00000000e+00,  121.88537547, 121.88537547),
       ( 0.00000000e+00,  0.00000000e+00,  -18.0319325 ,  18.0319325 ),
       ( 0.00000000e+00,  0.00000000e+00,  103.85344298, 139.91730797),
       (-2.84217094e-14, -2.44249065e-15,  121.88537547, 121.88537547),
       ( 2.84217094e-14,  2.88657986e-15,  -38.42187168,  38.42187168),
       (-1.83832924e+01, -1.79818313e+00,   97.94064938, 136.83882912)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

In [7]:
event.pdg[:8]  # pdg codes of the first 8 particles

array([ 2212, -2212,     1,    -1,    23,     1,    -1,    23],
      dtype=int32)

The length of these iterators is endless, so calling `len()` results in a `NotImplementedError`. However, LHE files can be used to generate data from hard processes stored on disk. In this case, the iterator has a length equal to the number of events in the LHE file. This can be determined using the `lhe` module.

In [8]:
shp.lhe.count_events("tt_bb_100.lhe.gz")

100

LHE events can be repeated or tiled using a `LheData` object, for studies which require to see the same event showered and hadronised many times.

In [14]:
lhe_data = shp.lhe.LheData.from_storage("tt_bb_100.lhe.gz")
lhe_data

LheData(num_events=100)

In [15]:
lhe_data.tile(2)

LheData(num_events=200)

In [16]:
lhe_data = shp.lhe.load_lhe("https://zenodo.org/record/6034610/files/unweighted_events.lhe.gz")

In [12]:
lhe_data.splitlines()[:10]

[b'<LesHouchesEvents version="3.0">',
 b'<header>',
 b'<!--',
 b'#*********************************************************************',
 b'#                                                                    *',
 b'#                        MadGraph5_aMC@NLO                           *',
 b'#                                                                    *',
 b'#                           Going Beyond                             *',
 b'#                                                                    *',
 b'#                   http://madgraph.hep.uiuc.edu                     *']

## Manipulating the event record with graphicle

In [58]:
import graphicle as gcl

**graphicle** is a portmanteau of "graph" and "particle", as it fuses the adjacency of the generation DAG with particle data.
Thin wrappers around the array data add semantic meaning and functionality to the attributes in the event record.
These data structures include `MomentumArray`, `PdgArray`, `AdjacencyList`, *etc*.

In [59]:
%%file pythia-settings.cmnd
PartonLevel:ISR = on
PartonLevel:FSR = on
PartonLevel:MPI = on
HadronLevel:all = on
Beams:frameType = 4

Overwriting pythia-settings.cmnd


In [60]:
gen = shp.generator.PythiaGenerator(
    "pythia-settings.cmnd",
    lhe_file="https://zenodo.org/record/6034610/files/unweighted_events.lhe.gz",
    rng_seed=1
)
event = next(gen)

 *-----------------------  SusyLesHouches SUSY/BSM Interface  ------------------------*
 | Last Change 12 Apr 2017 - P. Skands
 | Parsing: /tmp/tmpcbtl4995
 | (SLHA::readFile) line 334 - storing non-SLHA(2) block: yukawa
 *------------------------------------------------------------------------------------*


In [61]:
pmu = gcl.MomentumArray(event.pmu)
pmu

MomentumArray([[ 0.00000000e+00  0.00000000e+00  6.49999993e+03  6.50000000e+03]
               [ 0.00000000e+00  0.00000000e+00 -6.49999993e+03  6.50000000e+03]
               [ 0.00000000e+00  0.00000000e+00  2.99108832e+02  2.99108832e+02]
               ...
               [ 3.11737853e-01 -2.26242178e+00 -6.81890135e+00  7.19118546e+00]
               [ 9.37969381e-03 -3.57643723e-01 -7.98177924e-01  8.74691380e-01]
               [ 3.07652721e-02 -4.35641329e-02 -4.56023615e-02  7.01705851e-02]],
              dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

In [70]:
pmu.mass

array([9.38269999e-01, 9.38269999e-01, 0.00000000e+00, ...,
       1.59838115e-07, 2.78726032e-08, 3.94727752e-09])

In [65]:
pmu_rotate = pmu.shift_phi(0.5 * 3.14)
pmu_rotate

MomentumArray([[ 0.00000000e+00  0.00000000e+00  6.49999993e+03  6.50000000e+03]
               [ 0.00000000e+00  0.00000000e+00 -6.49999993e+03  6.50000000e+03]
               [ 0.00000000e+00  0.00000000e+00  2.99108832e+02  2.99108832e+02]
               ...
               [ 2.26266931e+00  3.09936128e-01 -6.81890135e+00  7.19118546e+00]
               [ 3.57651079e-01  9.09488958e-03 -7.98177924e-01  8.74691380e-01]
               [ 4.35886183e-02  3.07305711e-02 -4.56023615e-02  7.01705851e-02]],
              dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

In [66]:
pdgs = gcl.PdgArray(event.pdg)
pdgs

PdgArray([2212 2212   21 ...   22   22   22], dtype=int32)

In [68]:
pdgs.name

array(['p', 'p', 'g', ..., 'gamma', 'gamma', 'gamma'], dtype=object)

In [69]:
pdgs.charge

array([1., 1., 0., ..., 0., 0., 0.])

In [71]:
adj = gcl.AdjacencyList(event.edges)
adj

AdjacencyList([[   0   -1]
               [   0   -2]
               [  -6   -3]
               ...
               [-970  981]
               [-971  982]
               [-971  983]],
              dtype=[('src', '<i4'), ('dst', '<i4')])

In [72]:
print(adj.leaves)

MaskArray([False False False ...  True  True  True], dtype=bool)


In [73]:
print(np.array_equal(adj.leaves, event.final))

True


### Composite data structures

These custom types may then be used to form composite data structures, which enables the ability to subscript over the full event.
Additionally, methods and routines in other modules combine different attributes from the same data structure to create higher level operations.

### Instantiating

There are two\* composite data structures for the event record: `ParticleSet`, and `Graphicle` (which contains `ParticleSet` and `AdjacencyList`).
These composites can be instantiated by either passing numpy arrays for each attribute, passing existing constituent objects, or by passing an object which has the same interface as `PythiaEvent`.

\*There is also a composite structure called `MaskGroup`, which will be discussed in the section about querying the event.

In [74]:
graph = gcl.Graphicle.from_event(event)
graph

name,px,py,pz,energy,color,anticolor,helicity,status,final,src,dst
p,0.00E+00,0.00E+00,6.50E+03,6.50E+03,0,0,9,-12,False,0,-1
p,0.00E+00,0.00E+00,-6.50E+03,6.50E+03,0,0,9,-12,False,0,-2
g,0.00E+00,0.00E+00,2.99E+02,2.99E+02,503,502,1,-21,False,-6,-3
g,-0.00E+00,-0.00E+00,-5.99E+02,5.99E+02,501,503,1,-21,False,-7,-3
t,2.34E+02,-2.20E+01,-4.76E+02,5.58E+02,501,0,0,-22,False,-3,-4
...,...,...,...,...,...,...,...,...,...,...,...
gamma,1.30E-02,-1.30E+00,-3.24E+00,3.49E+00,0,0,9,91,True,-969,979
gamma,1.70E-01,-8.21E-01,-2.32E+00,2.47E+00,0,0,9,91,True,-970,980
gamma,3.12E-01,-2.26E+00,-6.82E+00,7.19E+00,0,0,9,91,True,-970,981
gamma,9.38E-03,-3.58E-01,-7.98E-01,8.75E-01,0,0,9,91,True,-971,982


The string representation of the composite data structures outputs a table, with columns for each attribute. The PDG, however, is replaced with the human-readable name of the particle.

The `src` and `dst` columns refer to the source and destination nodes of each edge that the particles represent.

### Querying the event record

One key advantage of this composite view is that it is possible to form complex filters over the whole event record.

A simple start is to use the `.hard_mask` attribute (actually aliased from `StatusArray.hard_mask`), which gives a boolean mask identifying the hard process.

In [75]:
graph[graph.hard_mask]

name,px,py,pz,energy,color,anticolor,helicity,status,final,src,dst
g,0.00E+00,0.00E+00,2.99E+02,2.99E+02,503,502,1,-21,False,-6,-3
g,-0.00E+00,-0.00E+00,-5.99E+02,5.99E+02,501,503,1,-21,False,-7,-3
t,2.34E+02,-2.20E+01,-4.76E+02,5.58E+02,501,0,0,-22,False,-3,-4
t~,-2.34E+02,2.20E+01,1.76E+02,3.40E+02,0,502,0,-22,False,-3,-5
b,1.43E+01,-5.00E+01,-1.38E+02,1.48E+02,501,0,-1,-23,False,-257,-329
W+,2.13E+02,2.50E+01,-3.42E+02,4.12E+02,0,0,0,-22,False,-257,-330
b~,-1.50E+02,3.20E+01,3.24E+01,1.56E+02,0,502,1,-23,False,-258,-345
W-,-8.66E+01,-1.10E+01,1.40E+02,1.83E+02,0,0,1,-22,False,-258,-346
tau+,4.02E+01,-1.04E+01,-2.41E+01,4.80E+01,0,0,9,-23,False,-333,-350
nu(tau),1.72E+02,3.52E+01,-3.18E+02,3.63E+02,0,0,9,-23,False,-333,-351


The `.hard_mask` attribute is actually a `MaskGroup`, which is a way of labelling and organising masks in hierarchies. `.hard_mask` is a flat `MaskGroup`, identifying different sections of the hard process, identified by status codes.

In [76]:
print(graph.hard_mask)

MaskGroup(agg_op=OR)
├── incoming
├── intermediate
├── outgoing
└── outgoing_nonperturbative_diffraction



To view only outgoing particles from the hard process, you can use string subscripting:

In [77]:
graph[graph.hard_mask["outgoing"]]

name,px,py,pz,energy,color,anticolor,helicity,status,final,src,dst
b,1.43E+01,-5.00E+01,-1.38E+02,1.48E+02,501,0,-1,-23,False,-257,-329
b~,-1.50E+02,3.20E+01,3.24E+01,1.56E+02,0,502,1,-23,False,-258,-345
tau+,4.02E+01,-1.04E+01,-2.41E+01,4.80E+01,0,0,9,-23,False,-333,-350
nu(tau),1.72E+02,3.52E+01,-3.18E+02,3.63E+02,0,0,9,-23,False,-333,-351
s,-4.85E+01,-4.58E+01,7.17E+01,9.80E+01,504,0,9,-23,False,-347,-355
c~,-3.81E+01,3.48E+01,6.81E+01,8.55E+01,0,504,9,-23,False,-347,-356


`MaskGroup` objects with a subset of the original masks can be created by passing multiple keys in a list.

In [78]:
sub_mask = graph.hard_mask[["outgoing", "intermediate"]]
print(sub_mask)
graph[sub_mask]

MaskGroup(agg_op=OR)
├── outgoing
└── intermediate



name,px,py,pz,energy,color,anticolor,helicity,status,final,src,dst
t,2.34E+02,-2.20E+01,-4.76E+02,5.58E+02,501,0,0,-22,False,-3,-4
t~,-2.34E+02,2.20E+01,1.76E+02,3.40E+02,0,502,0,-22,False,-3,-5
b,1.43E+01,-5.00E+01,-1.38E+02,1.48E+02,501,0,-1,-23,False,-257,-329
W+,2.13E+02,2.50E+01,-3.42E+02,4.12E+02,0,0,0,-22,False,-257,-330
b~,-1.50E+02,3.20E+01,3.24E+01,1.56E+02,0,502,1,-23,False,-258,-345
W-,-8.66E+01,-1.10E+01,1.40E+02,1.83E+02,0,0,1,-22,False,-258,-346
tau+,4.02E+01,-1.04E+01,-2.41E+01,4.80E+01,0,0,9,-23,False,-333,-350
nu(tau),1.72E+02,3.52E+01,-3.18E+02,3.63E+02,0,0,9,-23,False,-333,-351
s,-4.85E+01,-4.58E+01,7.17E+01,9.80E+01,504,0,9,-23,False,-347,-355
c~,-3.81E+01,3.48E+01,6.81E+01,8.55E+01,0,504,9,-23,False,-347,-356


Nesting `MaskGroup`s becomes very useful when defining the hierarchy of the shower descending from the hard process.
Routines in the `graphicle.select` module form more complex masks over the event record.
One such routine is `hierarchy()`, which identifies the partons in the hard process, and then traverses the DAG to the final state, identifying the descendant particles.

In [79]:
hier = gcl.select.hierarchy(graph)
print(hier)

MaskGroup(agg_op=OR)
├── t
│   ├── b
│   ├── W+
│   │   ├── tau+
│   │   ├── nu(tau)
│   │   └── latent
│   └── latent
└── t~
    ├── b~
    ├── W-
    │   ├── s
    │   ├── c~
    │   └── latent
    └── latent



In [86]:
tau_pmu = graph.pmu[hier["t"]["W+"]["tau+"] & graph.final]
len(tau_pmu)

7

graphicle's wrappers are transparent to numpy, so simple functions, such as a sum, can be applied with ease.

By applying a mask over the descendants of a parton in the final state, and then summing the momenta, the parton's momentum can be reconstructed.

In [87]:
tau_pmu_agg = np.sum(tau_pmu, axis=0)
tau_pmu_agg

MomentumArray([[ 40.18776094 -10.42345447 -24.10224852  48.03935895]], dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

In [88]:
tau_pmu_agg.mass

array([1.77682096])

In [91]:
hard_tau_mask = graph.hard_mask & (graph.pdg.name == "tau+")
graph[hard_tau_mask]

name,px,py,pz,energy,color,anticolor,helicity,status,final,src,dst
tau+,4.02E+01,-1.04E+01,-2.41E+01,4.80E+01,0,0,9,-23,False,-333,-350


The reconstructed mass can be checked against the reference value from the PDG record.

In [92]:
graph.pdg[hard_tau_mask].mass

array([1.77686])

If you go up one level of nesting in the `MaskGroup`, you can reconstruct the parent parton, the `W+`:

In [95]:
W_plus_pmu = graph.pmu[hier["t"]["W+"] & graph.final]
np.sum(W_plus_pmu, axis=0).mass

array([80.41900245])